### Data Prep

In [16]:
from datasets import load_dataset
ds = load_dataset("microsoft/cats_vs_dogs")

In [17]:
ds['train']
ds

DatasetDict({
    train: Dataset({
        features: ['image', 'labels'],
        num_rows: 23410
    })
})

In [18]:
from PIL import Image, ImageOps


In [19]:
def preprocessing(example):
    size = (64,64)
    img = example['image']
    img_resized = img.resize(size)
    example['image'] = img_resized
    return example

ds_resized = ds['train'].map(preprocessing, num_proc=4)


In [20]:
test = ds['train']['image'][0]

In [21]:
import numpy as np
test_flatten = test.getdata()
arr = np.array(test_flatten)
arr.flatten()


array([203, 164,  87, ...,   1,   1,   0])

In [22]:
def flattenization(example):
    import numpy as np
    img = example['image']
    img_processed = np.array(img, dtype=np.float32)/255
    example['image'] = img_processed.flatten()
    return example

ds_flatten = ds_resized.map(flattenization, num_proc=1)


In [23]:
ds_flatten.shape

(23410, 2)

In [24]:
def split_function(database, x):
    size_train = round(ds_flatten.shape[0]*x)
    split_train = database[:size_train]
    split_test = database[size_train:]
    return split_train, split_test

(split_train, split_test) = split_function(ds_flatten,x = 0.8)
split_test



{'image': [<PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
  <PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
  <PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
  <PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
  <PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
  <PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
  <PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
  <PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
  <PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
  <PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
  <PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
  <PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
  <PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
  <PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
  <PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
  <PIL.TiffImage

In [94]:
split_train['image']

[<PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
 <PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
 <PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
 <PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
 <PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
 <PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
 <PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
 <PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
 <PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
 <PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
 <PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
 <PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
 <PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
 <PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
 <PIL.TiffImagePlugin.TiffImageFile image mode=F size=1x12288>,
 <PIL.TiffImagePlugin.TiffImageFile imag

### NN 
Vectorization


In [ ]:
import numpy as np

def log_reg(z):
    return 1/(1+np.exp(-z))

class NN():
    def __init__(self,L,nx,alpha,activation_funcs): #only hidden layers
        self.L = L
        self.nx = nx 
        self.alpha = alpha
        self.activation_funcs = activation_funcs
    
    def randomize_weights(self,X0,m_example=1): #X0 ca doit être le nombre de units à N0
        W = np.empty(self.L,dtype=object)
        B = np.empty(self.L,dtype=object)
        W[0] = np.random.randn(self.nx[0],X0)
        B[0] = np.random.randn(self.nx[0],m_example)
        for l in range(1,self.L):
            W[l] = np.random.randn(self.nx[l],self.nx[l-1])
            B[l] = np.random.randn(self.nx[l],m_example)
        self.W = W
        self.B = B
        print('W : ', self.W)
        print('B :', self.B)
    
    
    def forward_propagation(self,X0m): 
        self.Z = np.empty(self.L,dtype=object)
        self.A = np.empty(self.L,dtype=object)
        self.Z[0] = np.dot(self.W[0],X0m)+self.B[0]
        print('Z0 is =', self.Z[0])
        self.A[0] = log_reg(self.Z[0])
        print('A0 is =', self.A[0])
        for l in range(1,self.L):
            self.Z[l] = np.dot(self.W[l],self.A[l-1]) + self.B[l]
            self.A[l] = log_reg(self.Z[l])
        print('A :', self.A)
    
    def compute_cost(self, m=1,A, Y):
        return 1/m*np.sum((A-Y)**2)
    


In [108]:
#quick test launch
this = NN(L = 6, nx = np.array([6,4,3,2,4,1]),alpha = .01,activation_funcs=log_reg)
this.randomize_weights(12288)
this.forward_propagation(split_train['image'][0]) # first forward propagation !!!
this.compute_cost(m = 1,A = this.A[5], Y = split_train['labels'][0])

W :  [array([[ 1.05027092,  0.9548631 , -0.11046949, ...,  0.69412285,
          0.82423795,  0.12337959],
        [ 0.32393844,  1.1327618 , -1.22203619, ...,  0.38784054,
          0.30946976,  2.0873328 ],
        [ 1.08588225, -0.22740895, -1.68418726, ...,  0.01190147,
          1.9132108 ,  0.10392162],
        [-1.84551156, -1.3193499 ,  1.73253632, ..., -0.21250356,
         -1.33052578,  1.30504182],
        [-0.85718145, -1.26284672, -1.15745537, ..., -0.41035072,
         -1.37187423, -1.25021161],
        [-0.03832679,  0.76103494,  0.36940623, ...,  0.3065245 ,
         -1.36728744,  0.12001335]])
 array([[-0.24546648,  2.04864387, -2.77369526,  0.97810818,  0.58831957,
         -0.90761851],
        [ 0.61906196, -0.12634403,  0.44844134, -0.26458988,  0.0485211 ,
         -1.61268511],
        [ 0.0619864 ,  1.06690036, -1.56224166,  0.64066022, -0.95960893,
         -0.35912022],
        [-0.77577134, -0.78573168, -1.3598536 ,  0.13943426,  0.85377176,
          0.46451

np.float64(0.002506144450809561)

W :  [array([[ 0.46517062,  0.31568492],
        [-1.52385197,  0.21637809],
        [-1.37509817, -2.31924442],
        [ 0.49958675, -0.11010136],
        [-0.23420727, -0.89049934]])
 array([[-0.3869043 , -0.38243667,  0.18192253,  0.47695394,  0.32463463],
        [-0.81376748, -0.00585717,  0.53529744, -0.76492255,  3.02742056]])]
B : [array([[ 0.17364659],
        [-1.20432541],
        [-0.07800597],
        [-0.61664113],
        [ 0.72279084]]) array([[ 0.25951241],
                               [-0.0416693 ]])]
